# Data Collection

Here we are collecting the appropriate data.
Since we are interested in virality of the movies, we will get movie data from the Movie Database, which has popularity measures. Then combine it to IMDB movie data and TN budget/revenue data.  

### The Movie Database (TMDB) 
It's a good open source alternative to IMDB.  
The documentation is here: https://www.themoviedb.org/documentation/api


#### Load the movie database API

In [426]:
import numpy as np
import pandas as pd

In [4]:
# Loading tmdb_api
# You must get your API key from themoviedb.org 
# and save your API key("api_key") in .secret/tmdb_api.json 

import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

path = '/Users/stereopickles/.secret' # input the location of your tmdb_api.json

api_key = get_keys(f"{path}/tmdb_api.json")['api_key']


In [6]:
# Collecting basic movie data with popularity and voting

import requests

url = 'https://api.themoviedb.org/3/discover/movie'

url_params = {
    "api_key": api_key,
    "sort_by": "popularity.desc", # sort by popularity
    "page": 1, # each page contains 20 movies
    "release_date.gte": '2000-01-01' 

}

resp = requests.get(url, params = url_params)
print(resp.status_code)
# if prints 200, we are good.

200


In [7]:
# Now that we reviewed the data type, 
# I'll run through each pages to create a full dataset 
# of top 10000 popular movies since 2000.

tmdb_dat = []

for i in range(500):
    url_params = {
        "api_key": api_key,
        "sort_by": "popularity.desc",
        "page": i+1,
        "release_date.gte": '2000-01-01' 
        #only movies with release date greater than 2000
    }
    resp = requests.get(url, params = url_params)
    try: 
        tmdb_dat.extend(resp.json()['results'])
    except:
        print(f"Error on page {i+1}")
        
tmdb_dat[0]    

{'popularity': 168.221,
 'vote_count': 3860,
 'video': False,
 'poster_path': '/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg',
 'id': 419704,
 'adult': False,
 'backdrop_path': '/t4z8OlOEzH7J1JRFUN3rcm6XHNL.jpg',
 'original_language': 'en',
 'original_title': 'Ad Astra',
 'genre_ids': [18, 878],
 'title': 'Ad Astra',
 'vote_average': 6.1,
 'overview': 'The near future, a time when both hope and hardships drive humanity to look to the stars and beyond. While a mysterious phenomenon menaces to destroy life on planet Earth, astronaut Roy McBride undertakes a mission across the immensity of space and its many perils to uncover the truth about a lost expedition that decades before boldly faced emptiness and silence in search of the unknown.',
 'release_date': '2019-09-17'}

In [25]:
movie_df = pd.DataFrame(tmdb_dat)
print(movie_df.shape)
movie_df.head(2)

(10000, 14)


,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,168.221,3860,False,/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg,419704,False,/t4z8OlOEzH7J1JRFUN3rcm6XHNL.jpg,en,Ad Astra,"[18, 878]",Ad Astra,6.1,"The near future, a time when both hope and har...",2019-09-17
1,197.612,104,False,/h1B7tW0t399VDjAcWJh8m87469b.jpg,556574,False,/uWVkEo9PWHu9algZsiLPi6sRU64.jpg,en,Hamilton,"[18, 36]",Hamilton,8.7,Presenting the tale of American founding fathe...,2020-07-03


In [26]:
# seems like some movies made before 2000 got in. Let's remove it.
movie_df = movie_df[movie_df.release_date >= '2000-01-01']

#### Additional cleaning
Since our dataframe is getting bigger, and it contains some information we will not be looking at, let's remove those columns. 

In [27]:
movie_df.drop(['video', 'poster_path','backdrop_path', 'original_title', 'genre_ids'], 
              axis = 1, inplace = True)

In [29]:
movie_df.head(2)

,popularity,vote_count,id,adult,original_language,title,vote_average,overview,release_date
0,168.221,3860,419704,False,en,Ad Astra,6.1,"The near future, a time when both hope and har...",2019-09-17
1,197.612,104,556574,False,en,Hamilton,8.7,Presenting the tale of American founding fathe...,2020-07-03


### Adding more information

Additionally, our questions need information on casting, budget, revenues, MPAA, etc.  
We need to get MPAA ratings for US, fetching separtely from TMDB. 

#### MPAA Rating
Lastly, I would like MPAA Rating. This is in the release_dates not called in the main movie tables. 

In [30]:
movie_df = movie_df.sort_values('id')

In [31]:
# reviewing MPAA data

url = f'https://api.themoviedb.org/3/movie/{movie_df.id[10]}/release_dates'

url_params = {
    "api_key": api_key
}

resp = requests.get(url, params = url_params)
print(resp.status_code)

200


In [32]:
# get information from each movie in movie_df (This will take a while)
MPAA = {}

for ind in movie_df.id:
    url = f'https://api.themoviedb.org/3/movie/{ind}/release_dates'
    url_params = {
        "api_key": api_key
    }
    resp = requests.get(url, params = url_params)
    res = resp.json()['results']
    try:
        MPAA[ind] = [x for x in res if x['iso_3166_1'] == 'US'][0]['release_dates'][0]['certification']
    except: 
        print(f"error on {ind}: {([x for x in res if x['iso_3166_1'] == 'US'])}")
        MPAA[ind] = np.nan
              
    print(f"{ind} complete / {movie_df.id[-1]}")
        

12 complete / 6303
16 complete / 6303
20 complete / 6303
22 complete / 6303
24 complete / 6303
25 complete / 6303
27 complete / 6303
35 complete / 6303
38 complete / 6303
55 complete / 6303
58 complete / 6303
59 complete / 6303
64 complete / 6303
65 complete / 6303
69 complete / 6303
70 complete / 6303
71 complete / 6303
74 complete / 6303
77 complete / 6303
79 complete / 6303
80 complete / 6303
82 complete / 6303
83 complete / 6303
86 complete / 6303
98 complete / 6303
107 complete / 6303
error on 113: []
113 complete / 6303
116 complete / 6303
118 complete / 6303
120 complete / 6303
121 complete / 6303
122 complete / 6303
129 complete / 6303
134 complete / 6303
141 complete / 6303
142 complete / 6303
146 complete / 6303
148 complete / 6303
153 complete / 6303
155 complete / 6303
161 complete / 6303
163 complete / 6303
167 complete / 6303
170 complete / 6303
176 complete / 6303
179 complete / 6303
180 complete / 6303
182 complete / 6303
186 complete / 6303
187 complete / 6303
189 comp

2176 complete / 6303
2179 complete / 6303
2185 complete / 6303
2196 complete / 6303
2207 complete / 6303
2239 complete / 6303
2251 complete / 6303
2252 complete / 6303
2253 complete / 6303
2260 complete / 6303
2266 complete / 6303
2267 complete / 6303
2268 complete / 6303
2269 complete / 6303
2270 complete / 6303
2274 complete / 6303
2284 complete / 6303
2288 complete / 6303
2289 complete / 6303
2294 complete / 6303
2295 complete / 6303
2309 complete / 6303
2310 complete / 6303
2312 complete / 6303
2313 complete / 6303
2332 complete / 6303
2334 complete / 6303
2335 complete / 6303
2355 complete / 6303
2359 complete / 6303
2395 complete / 6303
error on 2440: []
2440 complete / 6303
2447 complete / 6303
2454 complete / 6303
2486 complete / 6303
2501 complete / 6303
2502 complete / 6303
2503 complete / 6303
error on 2517: []
2517 complete / 6303
2539 complete / 6303
2567 complete / 6303
2577 complete / 6303
2610 complete / 6303
2637 complete / 6303
2642 complete / 6303
2652 complete / 630

8488 complete / 6303
8489 complete / 6303
8555 complete / 6303
8584 complete / 6303
8617 complete / 6303
8618 complete / 6303
8619 complete / 6303
8643 complete / 6303
8645 complete / 6303
error on 8653: []
8653 complete / 6303
8665 complete / 6303
8669 complete / 6303
8676 complete / 6303
8681 complete / 6303
8696 complete / 6303
8698 complete / 6303
8699 complete / 6303
8780 complete / 6303
8814 complete / 6303
8832 complete / 6303
8835 complete / 6303
8836 complete / 6303
8843 complete / 6303
8848 complete / 6303
8849 complete / 6303
8859 complete / 6303
8869 complete / 6303
8870 complete / 6303
8871 complete / 6303
8880 complete / 6303
8881 complete / 6303
8884 complete / 6303
8885 complete / 6303
8909 complete / 6303
8915 complete / 6303
8920 complete / 6303
8922 complete / 6303
8923 complete / 6303
error on 8937: []
8937 complete / 6303
8938 complete / 6303
8940 complete / 6303
8942 complete / 6303
8944 complete / 6303
8952 complete / 6303
8953 complete / 6303
8954 complete / 630

10200 complete / 6303
10201 complete / 6303
10202 complete / 6303
10204 complete / 6303
10214 complete / 6303
10226 complete / 6303
10229 complete / 6303
10247 complete / 6303
10253 complete / 6303
10258 complete / 6303
10263 complete / 6303
error on 10268: []
10268 complete / 6303
10288 complete / 6303
10292 complete / 6303
10294 complete / 6303
10296 complete / 6303
10304 complete / 6303
10313 complete / 6303
10315 complete / 6303
10316 complete / 6303
10317 complete / 6303
10320 complete / 6303
10327 complete / 6303
10330 complete / 6303
10358 complete / 6303
10360 complete / 6303
10361 complete / 6303
10362 complete / 6303
10367 complete / 6303
10375 complete / 6303
10378 complete / 6303
10383 complete / 6303
10384 complete / 6303
10385 complete / 6303
10389 complete / 6303
10391 complete / 6303
10393 complete / 6303
10416 complete / 6303
10425 complete / 6303
10426 complete / 6303
10429 complete / 6303
10431 complete / 6303
10461 complete / 6303
10468 complete / 6303
10471 complet

12085 complete / 6303
12090 complete / 6303
12093 complete / 6303
12094 complete / 6303
12096 complete / 6303
12100 complete / 6303
12103 complete / 6303
12107 complete / 6303
12109 complete / 6303
12113 complete / 6303
12133 complete / 6303
12138 complete / 6303
12140 complete / 6303
12142 complete / 6303
12146 complete / 6303
12149 complete / 6303
12153 complete / 6303
12155 complete / 6303
12162 complete / 6303
12163 complete / 6303
12171 complete / 6303
12177 complete / 6303
12178 complete / 6303
12180 complete / 6303
12182 complete / 6303
12184 complete / 6303
12192 complete / 6303
12193 complete / 6303
12201 complete / 6303
12205 complete / 6303
12211 complete / 6303
12212 complete / 6303
12222 complete / 6303
12225 complete / 6303
12242 complete / 6303
12244 complete / 6303
12246 complete / 6303
12253 complete / 6303
12271 complete / 6303
12277 complete / 6303
12279 complete / 6303
12289 complete / 6303
12308 complete / 6303
12312 complete / 6303
12400 complete / 6303
12403 comp

...

459258 complete / 6303
459840 complete / 6303
459910 complete / 6303
459928 complete / 6303
459947 complete / 6303
459992 complete / 6303
460019 complete / 6303
error on 460059: []
460059 complete / 6303
460071 complete / 6303
460229 complete / 6303
460279 complete / 6303
460321 complete / 6303
error on 460492: []
460492 complete / 6303
460555 complete / 6303
460648 complete / 6303
460668 complete / 6303
460790 complete / 6303
460793 complete / 6303
460846 complete / 6303
460885 complete / 6303
461053 complete / 6303
461054 complete / 6303
461130 complete / 6303
461297 complete / 6303
461928 complete / 6303
error on 462469: []
462469 complete / 6303
462650 complete / 6303
462883 complete / 6303
462919 complete / 6303
463053 complete / 6303
463088 complete / 6303
463257 complete / 6303
463272 complete / 6303
463319 complete / 6303
463322 complete / 6303
error on 463387: []
463387 complete / 6303
463602 complete / 6303
463684 complete / 6303
463821 complete / 6303
463843 complete / 6303


499726 complete / 6303
error on 500006: []
500006 complete / 6303
500475 complete / 6303
500535 complete / 6303
error on 500656: []
500656 complete / 6303
500664 complete / 6303
500682 complete / 6303
500852 complete / 6303
500860 complete / 6303
500916 complete / 6303
500919 complete / 6303
500921 complete / 6303
500922 complete / 6303
501170 complete / 6303
501395 complete / 6303
501630 complete / 6303
error on 501633: []
501633 complete / 6303
501907 complete / 6303
502143 complete / 6303
502166 complete / 6303
502292 complete / 6303
error on 502350: []
502350 complete / 6303
502406 complete / 6303
502416 complete / 6303
502422 complete / 6303
502425 complete / 6303
502682 complete / 6303
503346 complete / 6303
503403 complete / 6303
503616 complete / 6303
503619 complete / 6303
503706 complete / 6303
error on 503902: []
503902 complete / 6303
503917 complete / 6303
503919 complete / 6303
504172 complete / 6303
504251 complete / 6303
error on 504253: []
504253 complete / 6303
504321

error on 532620: []
532620 complete / 6303
532671 complete / 6303
532880 complete / 6303
error on 532938: []
532938 complete / 6303
533444 complete / 6303
533642 complete / 6303
533715 complete / 6303
error on 533992: []
533992 complete / 6303
534039 complete / 6303
534235 complete / 6303
534259 complete / 6303
534338 complete / 6303
534490 complete / 6303
534780 complete / 6303
error on 534928: []
534928 complete / 6303
error on 534939: []
534939 complete / 6303
535167 complete / 6303
535265 complete / 6303
535292 complete / 6303
error on 535356: []
535356 complete / 6303
535389 complete / 6303
535437 complete / 6303
535544 complete / 6303
535581 complete / 6303
536115 complete / 6303
536176 complete / 6303
error on 536506: []
536506 complete / 6303
536517 complete / 6303
536743 complete / 6303
536869 complete / 6303
537055 complete / 6303
537056 complete / 6303
537059 complete / 6303
537061 complete / 6303
537140 complete / 6303
537190 complete / 6303
error on 537526: []
537526 compl

585839 complete / 6303
error on 586333: []
586333 complete / 6303
586347 complete / 6303
586451 complete / 6303
586461 complete / 6303
586592 complete / 6303
586863 complete / 6303
586940 complete / 6303
587138 complete / 6303
587301 complete / 6303
587496 complete / 6303
587693 complete / 6303
587808 complete / 6303
error on 588001: []
588001 complete / 6303
error on 588009: []
588009 complete / 6303
588226 complete / 6303
589049 complete / 6303
589739 complete / 6303
error on 589982: []
589982 complete / 6303
error on 590009: []
590009 complete / 6303
590123 complete / 6303
590164 complete / 6303
590575 complete / 6303
error on 591121: []
591121 complete / 6303
591237 complete / 6303
591278 complete / 6303
591629 complete / 6303
591634 complete / 6303
591774 complete / 6303
error on 592041: []
592041 complete / 6303
592350 complete / 6303
error on 592480: []
592480 complete / 6303
error on 592717: []
592717 complete / 6303
592834 complete / 6303
592863 complete / 6303
592867 complete

KeyError: 'results'

In [33]:
# turning it to dataframe to join
MPAA_df = pd.DataFrame.from_dict(MPAA, orient = 'index', columns = ['MPAA'])

In [384]:
full_df = movie_df.join(MPAA_df, on = 'id')

Before merging other datasets, let's make some information consistent.

In [385]:
# make all title lower to match across database
full_df['title'] = full_df['title'].apply(lambda x: x.lower())

# add a year to match addition to movie title (in case of same titles)
full_df['year'] = full_df.release_date.apply(lambda x: x[0:4])
full_df['month'] = full_df.release_date.apply(lambda x: x[5:7])

Existing Bom_movie dataset has some information on studios, we'll add it just in case.

In [386]:
bom_gross = pd.read_csv('zippedData/bom.movie_gross.csv')
bom_gross['title'] = bom_gross['title'].apply(lambda x: x.lower())
len(bom_gross)

3387

In [387]:
# change bom_gross year to character to match other db
bom_gross.year = bom_gross.year.apply(str)
bom_gross = bom_gross[['title', 'year', 'studio']]

In [388]:
full_df = full_df.merge(bom_gross, right_on = ["title", "year"], \
                        left_on = ["title", "year"], how = 'left')

TN movie_budgets dataset has more extensive budget and gross information. Let's add these information

In [389]:
tn_budget = pd.read_csv('zippedData/tn.movie_budgets.csv')
tn_budget.head()
len(tn_budget)

5782

In [390]:
# make all title lower to match across database
tn_budget['movie'] = tn_budget['movie'].apply(lambda x: x.lower())
# just get year information to match
tn_budget['year'] = tn_budget.release_date.apply(lambda x: x[-4:])

In [391]:
# merge dataframes
full_df = full_df.merge(tn_budget, right_on = ["movie", "year"], \
                        left_on = ["title", "year"], how = 'left')

In [392]:
full_df.columns

Index(['popularity', 'vote_count', 'id_x', 'adult', 'original_language',
       'title', 'vote_average', 'overview', 'release_date_x', 'MPAA', 'year',
       'month', 'studio', 'id_y', 'release_date_y', 'movie',
       'production_budget', 'domestic_gross', 'worldwide_gross'],
      dtype='object')

In [393]:
# let's drop some leftover columns
full_df.drop(['release_date_x', 'release_date_y','id_y', 'movie'], axis = 1, inplace = True)

In [394]:
len(full_df)

8017

#### IMDB
Let's pull some more information from IMDB


In [395]:
# Loading IMDB datasets
imdb_dat = pd.read_csv('zippedData/imdb.title.basics.csv')
imdb_rating = pd.read_csv('zippedData/imdb.title.ratings.csv')
imdb_crew = pd.read_csv('zippedData/imdb.title.crew.csv')

In [396]:
# let's  filter down imdb_dat to match movies in full_df and merge. 
# We will prioritize IMDB information as we need the in depth info from IMDB


imdb_dat['primary_title'] = imdb_dat['primary_title'].apply(lambda x: x.lower())
imdb_dat['start_year'] = imdb_dat['start_year'].apply(lambda x: str(x))

full_df = full_df.merge(imdb_dat, right_on = ["primary_title", "start_year"], \
                        left_on = ["title", "year"])

In [397]:
len(full_df)

3878

#### IMDB Rating
Adding IMDB rating information

In [398]:
full_df = full_df.merge(imdb_rating, on = ['tconst'])
full_df.rename(columns={"averagerating": "avg_rating_imdb", "numvotes": "num_votes_imdb"}, inplace = True)


#### Directors & Writers
IMDB has info on directors and writers but they are in name codes  
I'll find the matching name from people database so we can understand who's who. 

In [399]:
# match imdb name codes for directors and writers to their actual names
imdb_people = pd.read_csv('zippedData/imdb.name.basics.csv')
pep_names = imdb_people.iloc[:, 0:2]

In [400]:
imdb_prin = pd.read_csv('zippedData/imdb.title.principals.csv')

In [401]:
# merge nconst to get actual names
imdb_prin_sub = imdb_prin.merge(pep_names, on = 'nconst')
imdb_prin_sub = imdb_prin_sub[['tconst', 'primary_name', 'category']]

In [402]:
actors = imdb_prin_sub[(imdb_prin_sub.category == 'actress') | (imdb_prin_sub.category == 'actor')][['tconst', 'primary_name']]
directors = imdb_prin_sub[imdb_prin_sub.category == 'director'][['tconst', 'primary_name']]
producers = imdb_prin_sub[imdb_prin_sub.category == 'producer'][['tconst', 'primary_name']]
editors = imdb_prin_sub[imdb_prin_sub.category == 'editor'][['tconst', 'primary_name']]
writers = imdb_prin_sub[imdb_prin_sub.category == 'writer'][['tconst', 'primary_name']]

LEVEL UP: If I have time - see if below can be done more elegantly using pivoting

In [403]:
actors = actors.groupby('tconst').primary_name.apply(lambda x: ','.join(x)).reset_index()
directors = directors.groupby('tconst').primary_name.apply(lambda x: ','.join(x)).reset_index()
producers = producers.groupby('tconst').primary_name.apply(lambda x: ','.join(x)).reset_index()
editors = editors.groupby('tconst').primary_name.apply(lambda x: ','.join(x)).reset_index()
writers = writers.groupby('tconst').primary_name.apply(lambda x: ','.join(x)).reset_index()


In [404]:
# merge
full_df = full_df.merge(actors, on = 'tconst', how = 'left')
full_df.rename(columns = {'primary_name':'actors'}, inplace = True)
full_df = full_df.merge(directors, on = 'tconst', how = 'left')
full_df.rename(columns = {'primary_name':'directors'}, inplace = True)
full_df = full_df.merge(producers, on = 'tconst', how = 'left')
full_df.rename(columns = {'primary_name':'producers'}, inplace = True)
full_df = full_df.merge(editors, on = 'tconst', how = 'left')
full_df.rename(columns = {'primary_name':'editors'}, inplace = True)
full_df = full_df.merge(writers, on = 'tconst', how = 'left')
full_df.rename(columns = {'primary_name':'writers'}, inplace = True)


## COCO 2017
There were 2 movies named Coco in 2017. This messes up the data since we have used year + title as identifying information for some of the data merge. so we will remove the horror coco.

In [405]:
full_df.drop(2332, inplace = True)

## Remove Duplicates
Duplicates got in somehow, let's remove them

In [406]:
full_df.drop_duplicates(subset = 'id_x', inplace= True)

## Cleaning up Genres

In [407]:
# add primary genre
full_df['primary_genre'] = full_df.genres.str.split(',', expand = True)[0]

In [408]:
full_df['secondary_genre'] = full_df.genres.str.split(',', expand = True)[1]

In [409]:
# function to call out animation as a separate genre
def newgen(x):
    if x.primary_genre == x.secondary_genre:
        return x.primary_genre
    if x.primary_genre == "Animation" or x.secondary_genre == "Animation":
        return "Animation"
    else:
        return x.primary_genre

In [412]:
full_df['new_genre'] = full_df.apply(newgen, axis = 1)

## profit_df
full_df contains all the information so far. Now I'll create sub-dataframes that keeps only budget/revenue information. If we run analyses that don't look at the budget/gross we can use full_df. Otherwise, we will use profit_df

In [413]:
# drop rows if any of the budget/gross information is missing
profit_df = full_df.dropna(axis = 0, 
                           subset = ['production_budget', 'domestic_gross', 'worldwide_gross'])

len(profit_df)

1236

In [414]:
# some has '$0' as its gross, which probably means that this data is not yet available. 
# let's exclude them if it's missing .
profit_df = profit_df[(profit_df.domestic_gross != '$0')]


In [415]:
# Now convert these information as numbers and scale it as a million $.
profit_df.production_budget = profit_df.production_budget.apply(lambda x: round(int(x[1:].replace(',', ''))/1000000, 2))
profit_df.domestic_gross = profit_df.domestic_gross.apply(lambda x: round(int(x[1:].replace(',', ''))/1000000, 2))
profit_df.worldwide_gross = profit_df.worldwide_gross.apply(lambda x: round(int(x[1:].replace(',', ''))/1000000, 2))

In [416]:
## Normalize worldwide_gross / domestic_gross (since their scales different 
# and we want to compare the level of success.

profit_df['worldwide_gross_z'] = profit_df.worldwide_gross.apply(
    lambda x: (x-profit_df.worldwide_gross.mean())/(profit_df.worldwide_gross.std()))
profit_df['domestic_gross_z'] = profit_df.domestic_gross.apply(
    lambda x: (x-profit_df.domestic_gross.mean())/(profit_df.domestic_gross.std()))

In [417]:
len(profit_df)

1193

In [427]:
profit_df.isnull().sum()

popularity              0
vote_count              0
id_x                    0
adult                   0
original_language       0
title                   0
vote_average            0
overview                0
MPAA                    1
year                    0
month                   0
studio                192
production_budget       0
domestic_gross          0
worldwide_gross         0
tconst                  0
primary_title           0
original_title          0
start_year              0
runtime_minutes         0
genres                  0
avg_rating_imdb         0
num_votes_imdb          0
actors                  7
directors              32
producers             135
editors              1048
writers               170
primary_genre           0
secondary_genre        67
new_genre               0
worldwide_gross_z       0
domestic_gross_z        0
dtype: int64

These missing categorical values are not key factors for all analyses. 
We will remove them as we see fit for individual topics.

In [419]:
#Exporting here to do other things

import csv
full_df.to_csv('data/full_data.csv')
profit_df.to_csv('data/profit_data.csv')

# Non-superhero data
Our data shows that last 10 years of movies have been highly influenced by superhero movies.  
But superhero movie market, dominated by big name studios, is a costly red ocean to break in.  
So we may want to look at the trend of film industry without these big name superhero films.  
We will exclude them by taking out films that had Jack Kirby, Stan Lee and Zak Penn as writers.  

In [420]:
profit_df_no_sup = profit_df[~ profit_df.writers.apply(lambda x: 'Jack Kirby' in str(x))]
profit_df_no_sup = profit_df_no_sup[~ profit_df_no_sup.writers.apply(lambda x: 'Stan Lee' in str(x))]
profit_df_no_sup = profit_df_no_sup[~ profit_df_no_sup.writers.apply(lambda x: 'Zak Penn' in str(x))]

In [421]:
profit_df_no_sup.sort_values(['worldwide_gross'], ascending = False).head(3)

,popularity,vote_count,id_x,adult,original_language,title,vote_average,overview,MPAA,year,...,actors,directors,producers,editors,writers,primary_genre,secondary_genre,new_genre,worldwide_gross_z,domestic_gross_z
1043,29.590,15745,135397,False,en,jurassic world,6.6,Twenty-two years after the events of Jurassic ...,PG-13,2015,...,"Chris Pratt,Bryce Dallas Howard,Ty Simpkins,Ju...",Colin Trevorrow,Patrick Crowley,NaN,"Rick Jaffa,Amanda Silver,Derek Connolly,Michae...",Action,Adventure,Action,5.960539,6.532597
1200,23.087,7612,168259,False,en,furious 7,7.3,Deckard Shaw seeks revenge against Dominic Tor...,PG-13,2015,...,"Dwayne Johnson,Vin Diesel,Jason Statham,Paul W...",James Wan,"Michael Fottrell,Neal H. Moritz",NaN,"Chris Morgan,Gary Scott Thompson",Action,Crime,Action,5.434842,3.176849
2290,43.033,7767,351286,False,en,jurassic world: fallen kingdom,6.5,Three years after the demise of Jurassic World...,,2018,...,"Rafe Spall,Chris Pratt,Bryce Dallas Howard,Jus...",J.A. Bayona,"Patrick Crowley,Belén Atienza",NaN,"Colin Trevorrow,Derek Connolly,Michael Crichton",Action,Adventure,Action,4.574571,3.902474


### Reassigning z-values (normalizing in this new population)


In [422]:
## Normalize worldwide_gross / domestic_gross (since their scales different 
# and we want to compare the level of success.

profit_df_no_sup['worldwide_gross_z'] = profit_df_no_sup.worldwide_gross.apply(
    lambda x: (x-profit_df_no_sup.worldwide_gross.mean())/(profit_df_no_sup.worldwide_gross.std()))
profit_df_no_sup['domestic_gross_z'] = profit_df_no_sup.domestic_gross.apply(
    lambda x: (x-profit_df_no_sup.domestic_gross.mean())/(profit_df_no_sup.domestic_gross.std()))

In [423]:
profit_df_no_sup.columns

Index(['popularity', 'vote_count', 'id_x', 'adult', 'original_language',
       'title', 'vote_average', 'overview', 'MPAA', 'year', 'month', 'studio',
       'production_budget', 'domestic_gross', 'worldwide_gross', 'tconst',
       'primary_title', 'original_title', 'start_year', 'runtime_minutes',
       'genres', 'avg_rating_imdb', 'num_votes_imdb', 'actors', 'directors',
       'producers', 'editors', 'writers', 'primary_genre', 'secondary_genre',
       'new_genre', 'worldwide_gross_z', 'domestic_gross_z'],
      dtype='object')

# Exporting 

In [424]:
#Exporting a new dataframe
profit_df_no_sup.to_csv('data/profit_data_no_sup.csv')